In [2]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
# import pyodbc
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('punkt')

In [ ]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [ ]:
articlesContent = execQuery("""Select content
from article""")

In [318]:
#connection = psycopg2.connect(user = "postgres",
#                                      password = "root",
#                                      host = "localhost",
#                                      port = "5432",
#                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random) 
filepath = '/home/daniel/OneDrive/KUuni/DataScience/Python/small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

In [53]:
import numpy as np
import nltk

df_1 = pd.DataFrame({'sent': [
                     'the sky is blue',
                     'the sky is not blue']})

series_1 = df_1['sent'].str.split()

# used to to tokenize
    # # Tokenize and generate count vectors
    # word_vec = series_of_lists_of_tokens.str.split().apply(pd.value_counts).fillna(0)


def tf_idf(series_of_lists_of_tokens):
    """ Returns tf-idf matrix(dataframe) for a given df-column/series with list of tokens for each row. """

    # Generate count vectors
    word_vec = series_of_lists_of_tokens.apply(pd.value_counts).fillna(0) # intuitive but slow


    # Compute term frequencies
        ### Assuming from http://www.tfidf.com/ -> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
    tf = word_vec.divide(np.sum(word_vec, axis='columns'), axis='index')

    # Compute inverse document frequencies
        ### Slides seems wrong s.18: https://docs.google.com/presentation/d/1rXhewxbmLX7mzDcy8aucI8OLIikfrLLiz8ocN5GS06I/edit#slide=id.g77508fe5d3_0_19
        ### Assuming from http://www.tfidf.com/ -> log_e(Total number of documents / Number of documents with term t in it)
    idf = np.log(len(tf) / word_vec[word_vec > 0].count()) 

    # Compute TF-IDF vectors and return
    tfidf = np.multiply(tf, idf.to_frame().T)
    
    return tfidf

# print(tf)
# print(idf)
# stemming = PorterStemmer()
test1 = df_1['sent'].iloc[0]
test2 = 'frightening, frightened frightens'
series_content_tokens = df_1['sent'].str.split()
# def tokenize(text):
#     list_tokens = nltk.word_tokenize(text)
#     a = [stemming.stem(word) for word in list_tokens]
#     return a

# df_tokens = series_content_tokens.explode().to_frame().reset_index().rename(columns={'index':'content_index'})
# df_tokens.groupby('content_index')[df_tokens.columns[1]].value_counts(normalize=True).unstack(level=1)


# tf_idf(series_content_tokens)
series_content_tokens
# df_tokens.apply(tuple).value_counts()

0         [the, sky, is, blue]
1    [the, sky, is, not, blue]
Name: sent, dtype: object

In [119]:
from nltk.tokenize import RegexpTokenizer
STEMMER = nltk.stem.SnowballStemmer('english')
TOKENIZER = RegexpTokenizer(r'\w+')
def textprocessing(text):
    return ''.join(STEMMER.stem(item)  for token in TOKENIZER.tokenize(text.lower()) if len(token) > 1)
a = 'frightening, frightened frightens'
textprocessing(a)

NameError: name 'item' is not defined